In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 6.6 MB/s 
     |████████████████████████████████| 6.6 MB 8.9 MB/s 
     |████████████████████████████████| 596 kB 48.4 MB/s 
     |████████████████████████████████| 86 kB 4.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:

from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

tokenizer = AutoTokenizer.from_pretrained("lserinol/bert-turkish-question-answering")
model = AutoModelForQuestionAnswering.from_pretrained("lserinol/bert-turkish-question-answering")

Downloading:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/245k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

In [ ]:
import nltk
nltk.download('punkt')

from nltk import tokenize

def seperate_sentences(text):
  return tokenize.sent_tokenize(text)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def answer_question(question, answer_text):
    '''
    Takes a `question` string and an `answer_text` string (which contains the
    answer), and identifies the words within the `answer_text` that are the
    answer. Prints them out.
    '''
    # ======== Tokenize ========
    # Apply the tokenizer to the input text, treating them as a text-pair.
    input_ids = tokenizer.encode(question, answer_text)

    # Report how long the input sequence is.
    print('Query has {:,} tokens.\n'.format(len(input_ids)))

    # ======== Set Segment IDs ========
    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizer.sep_token_id)

    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)

    # ======== Evaluate ========
    # Run our example through the model.
    outputs = model(torch.tensor([input_ids]), # The tokens representing our input text.
                    token_type_ids=torch.tensor([segment_ids]), # The segment IDs to differentiate question from answer_text
                    return_dict=True) 

    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    # ======== Reconstruct Answer ========
    # Find the tokens with the highest `start` and `end` scores.
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)

    # Get the string versions of the input tokens.
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # Start with the first token.
    answer = tokens[answer_start]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):
        
        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        
        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]

    return answer

In [ ]:
# answer_text ="Text: Kit İçeriği: Canon EOS M6 Mark II 15-45mm IS STM Fotoğraf Makinesi, Boya BY-MM1 Shotgun Mikrofon, Ulanzi MT-03 Masaüstü Tripod, SanDisk Ultra 32GB SDHC 100MB/s SD, Hafıza Kartı, Taşıma Çantası. Yaşamlarını spontane, yaratıcı fotoğrafçılık ve film yapımı ile belgelendirmekten hoşlanan herkes için Canon EOS M6 Mark II, son derece taşınabilir bir tasarımda hız, güç ve çok yönlülük sağlar. Manzara, portre ve sokak fotoğrafçılığı için mükemmeldir. Canon EOS M6 Mark II Özellikler: 32,5 Megapiksel, APS-C CMOS Sensör, Canon EF-M Lenslerle Uyumlu, 4K UHD Video Kayıt, 7.5cm 180° Eğilebilir Dokunmatik LCD Ekran, 5481 AF Puanlı Çift Piksel CMOS AF, 3.5mm Mikrofon Girişi, Dahili Wi-Fi / Bluetooth, Ağırlık-408g (Gövde). Canon EF-M 15-45mm Lens Özellikler: Canon EF-M Mount / APS-C Sensör Uyumlu, 24-72mm (35mm Eşdeğeri), Maksimum Diyafram Aralığı: - f / 3.5-6.3 , Üç küresel elemanlar, STM odaklama, Görüntü Sabitleyici, Minimum Odak Mesafesi: 25cm, Otomatik odaklama, Filtre Çapı: 49mm, Lens Ağırlığı: 130g. Çekim Ortamları: Portre, Manzara, Günlük Kullanım. Menü Dilleri: İngilizce, Almanca, Fransızca, Felemenkçe, Danca, Portekizce, Fince, İtalyanca, Ukrayna, Norveççe, İsveççe, İspanyolca, Yunanca, Rusça, Lehçe, Çekçe, Macarca, Vietnamca, Hintçe, Romence, Türkçe, Arapça, Tayca, Basitleştirilmiş Çince , Geleneksel Çince, Korece, Malayca, Endonezyaca ve Japonca. "

answer_text = "Canon EVF-DC2 elektronik vizör, makinenin aksesuar yuvasına takılır ve net, yüksek çözünürlüklü göz seviyesinde görüntüleme için 0,39 inç (0.99cm)  2,36m noktalı OLED ekrana sahiptir.Ayrıca, dahili dioptrik ayar sunar ve Gözlük takanlara uygun 22mm göz mesafesi sağlar. DIGIC 8 Görüntü İşlemcisi, düşük ışıkta bile yüksek görüntü kalitesi ve hızlı işlem üretmeye yardımcı olur. Yüksek ISO ayarlarını kullanırken, görüntü işlemcisi, neredeyse her türlü aydınlatma koşulunda sonuçların keskin ve ayrıntılı kalmasına yardımcı olur. Canon EOS M6 Mark II nin yeni Yüksek Hızlı Kesintisiz + (H +) çekim moduyla gerçekleşen aksiyonun hiçbir anını asla kaçırmayın. 14 fps'ye kadar (AF / AE İzleme ile) sürekli çekim yapabilme özelliğiyle, neredeyse hiç zaman kaybetmeden görüntüler yakalayabilirsiniz. EOS M6 Mark II kamera, istediğiniz fotoğrafı anında yakalamanıza yardımcı olan hızlı, doğru otomatik netleme için Dual Pixel CMOS AF ile çekim yapmanızı sağlar.Canon EOS M6 Mark II fotoğraf makinesi kullanılarak ön çekimle RAW seri çekim modu artık mümkün. AF izleme ile saniyede 30 kareye kadar çekim yapmanın keyfini çıkarın. RAW seri çekim modu, deklanşöre tam olarak basmadan önce size 0,5 saniyelik ön çekim görüntüleri sağlayan bir özellik içerir. Bu, mükemmel anı asla kaçırmamanızı sağlayacaktır. Canon EOS M6 Mark II, Wi-Fi ağ bağlantılarını hızlı ve kolay hale getirmek için tasarlanmıştır. Camera Connect uygulamasını kullanarak dosyaları doğrudan uyumlu bir akıllı cihaza aktarabilir. Bluetooth eşleştirme, ücretsiz Canon Camera Connect uygulamasını kullanarak fotoğraf makinesini uyumlu akıllı cihazlara bağlamanıza yardımcı olur. Bluetooth işlevini özelleştirmek için, LCD ekranda bulunan ağ simgesine tıklamanız yeterlidir. Ayrıca Bluetooth, uzaktan çekim için isteğe bağlı Kablosuz Uzaktan Kumanda BR-E1'e bağlanmanıza ve ayrıca kullanıcının uyumlu akıllı telefonundan GPS çekim konumu verilerini almanıza olanak tanır. Canon EVF-DC2 elektronik vizör, makinenin aksesuar yuvasına takılır ve net, yüksek çözünürlüklü göz seviyesinde görüntüleme için 0,39 inç (0.99cm)  2,36m noktalı OLED ekrana sahiptir.Ayrıca, dahili dioptrik ayar sunar ve Gözlük takanlara uygun 22mm göz mesafesi sağlar. DIGIC 8 Görüntü İşlemcisi, düşük ışıkta bile yüksek görüntü kalitesi ve hızlı işlem üretmeye yardımcı olur. Yüksek ISO ayarlarını kullanırken, görüntü işlemcisi, neredeyse her türlü aydınlatma koşulunda sonuçların keskin ve ayrıntılı kalmasına yardımcı olur. Canon EOS M6 Mark II nin yeni Yüksek Hızlı Kesintisiz + (H +) çekim moduyla gerçekleşen aksiyonun hiçbir anını asla kaçırmayın. 14 fps'ye kadar (AF / AE İzleme ile) sürekli çekim yapabilme özelliğiyle, neredeyse hiç zaman kaybetmeden görüntüler yakalayabilirsiniz. EOS M6 Mark II kamera, istediğiniz fotoğrafı anında yakalamanıza yardımcı olan hızlı, doğru otomatik netleme için Dual Pixel CMOS AF ile çekim yapmanızı sağlar.Canon EOS M6 Mark II fotoğraf makinesi kullanılarak ön çekimle RAW seri çekim modu artık mümkün. AF izleme ile saniyede 30 kareye kadar çekim yapmanın keyfini çıkarın. RAW seri çekim modu, deklanşöre tam olarak basmadan önce size 0,5 saniyelik ön çekim görüntüleri sağlayan bir özellik içerir. Bu, mükemmel anı asla kaçırmamanızı sağlayacaktır. Canon EOS M6 Mark II, Wi-Fi ağ bağlantılarını hızlı ve kolay hale getirmek için tasarlanmıştır. Camera Connect uygulamasını kullanarak dosyaları doğrudan uyumlu bir akıllı cihaza aktarabilir. Bluetooth eşleştirme, ücretsiz Canon Camera Connect uygulamasını kullanarak fotoğraf makinesini uyumlu akıllı cihazlara bağlamanıza yardımcı olur. Bluetooth işlevini özelleştirmek için, LCD ekranda bulunan ağ simgesine tıklamanız yeterlidir. Ayrıca Bluetooth, uzaktan çekim için isteğe bağlı Kablosuz Uzaktan Kumanda BR-E1'e bağlanmanıza ve ayrıca kullanıcının uyumlu akıllı telefonundan GPS çekim konumu verilerini almanıza olanak tanır."
quest = "sadece çantayı satın alabilir miyim?"

answer_question(quest, answer_text)


Token indices sequence length is longer than the specified maximum sequence length for this model (820 > 512). Running this sequence through the model will result in indexing errors


Query has 820 tokens.



RuntimeError: ignored

# Visualization 


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


def start_scores_plot(start_scores, tokens):
  # Use plot styling from seaborn.
  sns.set(style='darkgrid')
  sns.set(font_scale=1.5)
  plt.rcParams["figure.figsize"] = (16,8)

  # Pull the scores out of PyTorch Tensors and convert them to 1D numpy arrays.
  s_scores = start_scores.detach().numpy().flatten()

  # We'll use the tokens as the x-axis labels. In order to do that, they all need
  # to be unique, so we'll add the token index to the end of each one.
  token_labels = []
  for (i, token) in enumerate(tokens):
    token_labels.append('{:} - {:>2}'.format(token, i))

  # Create a barplot showing the start word score for all of the tokens.
  ax = sns.barplot(x=token_labels, y=s_scores, ci=None)

  # Turn the xlabels vertical.
  ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha="center")

  # Turn on the vertical grid to help align words to scores.
  ax.grid(True)

  plt.title('Start Word Scores')
  plt.savefig('Start_Word_Scores.png', bbox_inches='tight')


def end_score(end_scores, tokens):
  # Use plot styling from seaborn.
  sns.set(style='darkgrid')
  sns.set(font_scale=1.5)
  plt.rcParams["figure.figsize"] = (16,8)

  # Pull the scores out of PyTorch Tensors and convert them to 1D numpy arrays.
  e_scores = end_scores.detach().numpy().flatten()

  # We'll use the tokens as the x-axis labels. In order to do that, they all need
  # to be unique, so we'll add the token index to the end of each one.
  token_labels = []
  for (i, token) in enumerate(tokens):
    token_labels.append('{:} - {:>2}'.format(token, i))

  # Create a barplot showing the end word score for all of the tokens.
  ax = sns.barplot(x=token_labels, y=e_scores, ci=None)

  # Turn the xlabels vertical.
  ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha="center")

  # Turn on the vertical grid to help align words to scores.
  ax.grid(True)

  plt.title('End Word Scores')
  plt.savefig('End_Word_Scores.png', bbox_inches='tight')


def scores(tokens, start_scores, end_scores):

  # We'll use the tokens as the x-axis labels. In order to do that, they all need
  # to be unique, so we'll add the token index to the end of each one.
  token_labels = []
  for (i, token) in enumerate(tokens):
    token_labels.append('{:} - {:>2}'.format(token, i))

  # Pull the scores out of PyTorch Tensors and convert them to 1D numpy arrays.
  s_scores = start_scores.detach().numpy().flatten()
  e_scores = end_scores.detach().numpy().flatten()

  # Store the tokens and scores in a DataFrame. 
  # Each token will have two rows, one for its start score and one for its end
  # score. The "marker" column will differentiate them. A little wacky, I know.
  scores = []
  for (i, token_label) in enumerate(token_labels):
    # Add the token's start score as one row.
    scores.append({'token_label': token_label, 
                   'score': s_scores[i],
                   'marker': 'start'})
    
    # Add  the token's end score as another row.
    scores.append({'token_label': token_label, 
                   'score': e_scores[i],
                   'marker': 'end'})
    
  df_scores = pd.DataFrame(scores)

  # Draw a grouped barplot to show start and end scores for each word.
  # The "hue" parameter is where we tell it which datapoints belong to which
  # of the two series.
  g = sns.catplot(x="token_label", y="score", hue="marker", data=df_scores,
                kind="bar", height=6, aspect=4)

  # Turn the xlabels vertical.
  g.set_xticklabels(g.ax.get_xticklabels(), rotation=90, ha="center")

  # Turn on the vertical grid to help align words to scores.
  g.ax.grid(True)

  fig = g.get_figure()
  fig.savefig("Start_and_End_Scores.png") 

In [ ]:
!pip freeze

absl-py==1.1.0
alabaster==0.7.12
albumentations==0.1.12
altair==4.2.0
appdirs==1.4.4
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
arviz==0.12.1
astor==0.8.1
astropy==4.3.1
astunparse==1.6.3
atari-py==0.2.9
atomicwrites==1.4.0
attrs==21.4.0
audioread==2.1.9
autograd==1.4
Babel==2.10.2
backcall==0.2.0
beautifulsoup4==4.6.3
bleach==5.0.0
blis==0.7.7
bokeh==2.3.3
branca==0.5.0
bs4==0.0.1
CacheControl==0.12.11
cached-property==1.5.2
cachetools==4.2.4
catalogue==2.0.7
certifi==2022.6.15
cffi==1.15.0
cftime==1.6.0
chardet==3.0.4
charset-normalizer==2.0.12
click==7.1.2
cloudpickle==1.3.0
cmake==3.22.5
cmdstanpy==0.9.5
colorcet==3.0.0
colorlover==0.3.0
community==1.0.0b1
contextlib2==0.5.5
convertdate==2.4.0
coverage==3.7.1
coveralls==0.5
crcmod==1.7
cufflinks==0.17.3
cvxopt==1.2.7
cvxpy==1.0.31
cycler==0.11.0
cymem==2.0.6
Cython==0.29.30
daft==0.0.4
dask==2.12.0
datascience==0.10.6
debugpy==1.0.0
decorator==4.4.2
defusedxml==0.7.1
descartes==1.1.0
dill==0.3.5.1
distributed==1.25.3
dlib==19